In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
from sklearn.metrics import mean_squared_error, mean_absolute_error
import dgl
import dgl.function as fn
import time

gcn_msg = fn.copy_src(src='h', out='m')
gcn_reduce = fn.sum(msg='m', out='h')

device = torch.device('cuda')

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def compute_metric(y_true, y_pred, name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse= np.sqrt(mean_squared_error(y_true, y_pred))
    mape = mean_absolute_percentage_error(y_true[np.where(y_true > 10)[0]], y_pred[np.where(y_true > 10)[0]])
    print("mae " + name, mae)    
    print("rmse " + name, rmse)
    print("mape(>10) " + name, mape)  
    return rmse

def build_graph(src, dst, nb_nodes, device):
    g = dgl.DGLGraph()
    g.add_nodes(nb_nodes)
    g.add_edges(src, dst)
    return g.to(device)

def create_batches(node_features_batch, batch_size, src, dst, nb_nodes, device):
    my_graphs = []
    for i in range(batch_size):
        temp_g = build_graph(src, dst, nb_nodes, device)
        temp_g.ndata['h'] = torch.from_numpy(node_features_batch[:,i,:]).to(device)
        my_graphs.append(temp_g)
    return dgl.batch(my_graphs)


class NodeApplyModule(nn.Module):
    def __init__(self, in_feats, out_feats, activation):
        super(NodeApplyModule, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)
        self.activation = activation
        
    def forward(self, node):
        h = self.linear(node.data['h'])
        h = self.activation(h)
        return {'h' : h}
    

class GCN(nn.Module):
    def __init__(self, in_feats, out_feats, activation):
        super(GCN, self).__init__()
        self.apply_mod = NodeApplyModule(in_feats, out_feats, activation)
        
    def forward(self, g, feature):
        g.ndata['h'] = feature
        g.update_all(gcn_msg, gcn_reduce)
        g.apply_nodes(func=self.apply_mod)
        return g.ndata.pop('h')

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

Using backend: pytorch


In [2]:
class MC_STGCN(nn.Module):
    def __init__(self, in_dim, hidden_dim, batch_size, community_detail, device, nb_neurons_gru=20, nb_neurons_dense=10):
        super().__init__()
        self.batch_size = batch_size
        self.device = device
        self.community_detail = community_detail
        self.nb_nodes = sum([len(_) for _ in community_detail])
        self.grus = nn.ModuleList([nn.GRU(1, 20, 2, batch_first=True) for _ in range(self.nb_nodes)])

        self.gcnlayers1 = nn.ModuleList([
            GCN(in_dim, 32, F.relu),
            GCN(32, hidden_dim, F.relu)
        ])
        
        self.gcnlayers2 = nn.ModuleList([
            GCN(in_dim, 32, F.relu),
            GCN(32, hidden_dim, F.relu)
        ])

        self.fcs = nn.ModuleList([nn.ModuleList([nn.Linear(hidden_dim*2*len(one_community), 100), nn.Linear(100, len(one_community))])
                                     for one_community in community_detail])
        
    def forward(self, g, g2):
        h = g.ndata['h']        
        h = h.unsqueeze(dim=2)

        gru_outputs = [gru(h[range(i, i+self.nb_nodes*self.batch_size, self.nb_nodes),:])[0][:,-1,:]
             for i, gru in enumerate(self.grus)]
        
        h = torch.zeros((self.nb_nodes*self.batch_size, 20)).to(self.device)

        for i in range(len(gru_outputs)):
            h[range(i, i+self.nb_nodes*self.batch_size, self.nb_nodes),:] = gru_outputs[i]

        h2 = h.clone().detach()
        
        for conv in self.gcnlayers1:
            h = conv(g, h)
        g.ndata['h'] = h

        for conv in self.gcnlayers2:
            h2 = conv(g2, h2)
        g2.ndata['h2'] = h2
        
        h3 = torch.cat([h, h2], dim=1)  # (nb_nodes * batch_size, 40)
        outputs = []
        for i in range(len(self.community_detail)):
            X = torch.cat([h3[range(j, j+self.nb_nodes*self.batch_size, self.nb_nodes),:] for j in self.community_detail[i]], dim=1)
            X = self.fcs[i][1](F.relu(self.fcs[i][0](X)))
            outputs.append(X)
        return outputs

In [3]:
def train(X_train, X_val, y_train, y_val, saved_path, nb_nodes, community_detail, high_similar_poi, edges, batch_size = 72, 
          learning_rate=0.001, epochs=100, device=device):
    
    nb_train, nb_val = y_train.shape[1], y_val.shape[1]
    pred_train, pred_val = np.zeros_like(y_train), np.zeros_like(y_val)
    
    net = MC_STGCN(20, 20, batch_size, community_detail, device).to(device)
    
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
    
    nb_nodes = sum([len(_) for _ in community_detail])
    
    early_stopping = EarlyStopping(patience=7, verbose=False, path=saved_path)
    
    for epoch in range(epochs):
        loss_all = []
        net.train()
        for i in range(nb_train//batch_size):
            batched_graph1 = create_batches(X_train[:,i*batch_size:(i+1)*batch_size,:], batch_size, edges[:,0], edges[:,1], nb_nodes, device)
            batched_graph2 = create_batches(X_train[:,i*batch_size:(i+1)*batch_size,:], batch_size, high_similar_poi['i'], high_similar_poi['j'], nb_nodes, device)
            logits_train = net(batched_graph1, batched_graph2) # [(72, 10), (72, 13), ...]
            y_train_temp = torch.from_numpy(y_train[:,i*batch_size:(i+1)*batch_size].T).to(device)

            for j in range(len(community_detail)):
                pred_train[community_detail[j], i*batch_size:(i+1)*batch_size] = logits_train[j].detach().cpu().numpy().T

            loss = sum([F.mse_loss(logits_train[j], y_train_temp[:, community_detail[j]], reduction='mean') for j in range(len(community_detail))])

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_all.append(loss.item() / 8)    
        loss_all = np.sqrt(np.sum(np.array(loss_all).mean())) 
        
        net.eval()
        with torch.no_grad():
            for i in range(nb_val//batch_size):
                batched_graph1 = create_batches(X_val[:,i*batch_size:(i+1)*batch_size,:], batch_size, edges[:,0], edges[:,1], nb_nodes, device)
                batched_graph2 = create_batches(X_val[:,i*batch_size:(i+1)*batch_size,:], batch_size, high_similar_poi['i'], high_similar_poi['j'], nb_nodes, device)
                logits_val = net(batched_graph1, batched_graph2)

                for j in range(len(community_detail)):
                    pred_val[community_detail[j], i*batch_size:(i+1)*batch_size] = logits_val[j].cpu().numpy().T
            rmse_current = compute_metric(y_val.flatten(), pred_val.flatten(), str(epoch))
            print()
        
        early_stopping(rmse_current, net)

        if early_stopping.early_stop:
            print('Early stopping')
            break

    net.load_state_dict(torch.load(saved_path))
    return net
 

## TaxiSZ

In [4]:
X_train_sz = np.load('./data/taxi_sz/X_train20.npy').astype('float32')
X_val_sz = np.load('./data/taxi_sz/X_val20.npy').astype('float32')
X_test_sz = np.load('./data/taxi_sz/X_test20.npy').astype('float32')
y_train_sz = np.load('./data/taxi_sz/y_train20.npy').astype('float32')
y_val_sz = np.load('./data/taxi_sz/y_val20.npy').astype('float32')
y_test_sz = np.load('./data/taxi_sz/y_test20.npy').astype('float32')

sz_8community = np.load('./data/taxi_sz/sz_8community.npy', allow_pickle=True).tolist()
print(sz_8community)

edges_in_sz = np.load('./data/taxi_sz/edges_in_sz.npy')
print(edges_in_sz.shape)

high_similar_poi_sz = pd.read_csv('./data/taxi_sz/high_similar_poi.csv')
high_similar_poi_sz.head(2)

[[0, 1, 2, 3, 4, 6, 9, 63, 69, 77], [5, 7, 8, 10, 22, 25, 45, 46, 47, 48, 49, 50, 51, 58, 66, 71, 81], [11, 21, 24, 29, 30, 35, 37, 39, 40, 41, 42, 43, 53, 92, 96], [12, 13, 15, 16, 23, 34, 36, 54, 55, 72, 78, 79, 97], [14, 17, 18, 19, 20, 38, 44, 84, 85, 86, 87, 88, 89, 90, 91, 93, 94, 95, 98, 100], [26, 27, 28, 31, 32, 33], [52, 56, 57, 59, 60, 61, 62, 64, 65], [67, 68, 70, 73, 74, 75, 76, 80, 82, 83, 99]]
(654, 2)


,i,j,pearson
0,1,4,0.919318
1,1,5,0.895695


In [18]:
model_sz =  train(X_train_sz, X_val_sz, y_train_sz, y_val_sz, "./saved/mc_stgcn_sz.pt", 101, sz_8community, high_similar_poi_sz, edges_in_sz, batch_size = 72, learning_rate=0.001, epochs=200, device=device)

mae 0 13.05255
rmse 0 21.357359
mape(>10) 0 42.48224496841431

mae 1 10.6705265
rmse 1 18.474245
mape(>10) 1 34.28148627281189

mae 2 10.196814
rmse 2 17.841137
mape(>10) 2 32.75590538978577

mae 3 10.621518
rmse 3 17.721306
mape(>10) 3 35.20554006099701

mae 4 11.248854
rmse 4 17.963436
mape(>10) 4 38.060006499290466

EarlyStopping counter: 1 out of 7
mae 5 10.900261
rmse 5 17.382298
mape(>10) 5 37.46967017650604

mae 6 9.791938
rmse 6 16.398085
mape(>10) 6 32.5910747051239

mae 7 8.998944
rmse 7 15.761854
mape(>10) 7 30.15967309474945

mae 8 9.170155
rmse 8 15.86978
mape(>10) 8 29.81162667274475

EarlyStopping counter: 1 out of 7
mae 9 8.960138
rmse 9 15.810209
mape(>10) 9 28.900697827339172

EarlyStopping counter: 2 out of 7
mae 10 9.332806
rmse 10 16.420723
mape(>10) 10 30.142158269882202

EarlyStopping counter: 3 out of 7
mae 11 9.551583
rmse 11 16.343622
mape(>10) 11 30.105268955230713

EarlyStopping counter: 4 out of 7
mae 12 9.173488
rmse 12 15.822946
mape(>10) 12 31.8765670061

NameError: name 'model' is not defined

In [5]:
nb_test, pred_test = y_test_sz.shape[1], np.zeros_like(y_test_sz)
model = MC_STGCN(20, 20, nb_test, sz_8community, device).to(device)
model.load_state_dict(torch.load("./saved/mc_stgcn_sz.pt"))

t0 = time.time()
with torch.no_grad():
    batched_graph1 = create_batches(X_test_sz, nb_test, edges_in_sz[:,0], edges_in_sz[:,1], 101, device)
    batched_graph2 = create_batches(X_test_sz, nb_test, high_similar_poi_sz['i'], high_similar_poi_sz['j'], 101, device)
    logits_test = model(batched_graph1, batched_graph2)
    print('Time cost: ', time.time() - t0)                        
    for j in range(len(sz_8community)):
        pred_test[sz_8community[j], :] = logits_test[j].cpu().numpy().T

rmse_test = compute_metric(y_test_sz.flatten(), pred_test.flatten(), 'TEST')

Time cost:  1.5964324474334717
mae TEST 6.468648
rmse TEST 11.005166
mape(>10) TEST 21.101422607898712


# TaxiNY

In [7]:
X_train_ny = np.load('./data/taxi_ny/X_train20.npy')
y_train_ny = np.load('./data/taxi_ny/y_train20.npy')
X_val_ny = np.load('./data/taxi_ny/X_val20.npy')
y_val_ny = np.load('./data/taxi_ny/y_val20.npy')
X_test_ny = np.load('./data/taxi_ny/X_test20.npy')
y_test_ny = np.load('./data/taxi_ny/y_test20.npy')

ny_6community = [[0, 5, 7, 10, 11, 12, 13, 32, 33, 49, 54], [1, 2, 4, 6, 21, 22, 24, 25, 43, 48, 56, 57, 59, 60, 61, 62],
    [3, 8, 9, 15, 16, 28, 29, 30, 34, 55],  [14, 17, 31, 36, 37, 38, 39, 41, 46, 47, 50, 52, 53], [18, 26, 42, 45, 51], 
    [19, 20, 23, 27, 35, 40, 44, 58]]

edges_in_ny = np.load('./data/taxi_ny/edges_manhattan.npy')
ny_high_similar_poi = pd.read_csv('./data/taxi_ny/high_similar_poi.csv')

print(X_train_ny.shape, y_train_ny.shape)

(63, 8856, 24) (63, 8856)


In [6]:
model_ny =  train(X_train_ny, X_val_ny, y_train_ny, y_val_ny, "./saved/mc_stgcn_ny.pt", 63, ny_6community, 
                  ny_high_similar_poi, edges_in_ny, batch_size = 72, learning_rate=0.0005, epochs=200, device=device)

mae 0 7.908276
rmse 0 19.041483
mape(>10) 0 36.10996603965759

mae 1 6.199063
rmse 1 14.923988
mape(>10) 1 29.23388183116913

mae 2 5.1178346
rmse 2 11.686999
mape(>10) 2 24.40401315689087

mae 3 4.7104697
rmse 3 10.157001
mape(>10) 3 22.901651263237

mae 4 4.521783
rmse 4 9.571823
mape(>10) 4 21.624548733234406

mae 5 4.432323
rmse 5 9.340339
mape(>10) 5 20.85219919681549

mae 6 4.3826976
rmse 6 9.253427
mape(>10) 6 20.471931993961334

mae 7 4.3108974
rmse 7 9.100637
mape(>10) 7 20.142734050750732

mae 8 4.2281485
rmse 8 8.955422
mape(>10) 8 19.7953999042511

mae 9 4.158102
rmse 9 8.83654
mape(>10) 9 19.565702974796295

mae 10 4.10677
rmse 10 8.739718
mape(>10) 10 19.41572278738022

mae 11 4.079119
rmse 11 8.674964
mape(>10) 11 19.301733374595642

mae 12 4.054496
rmse 12 8.6100025
mape(>10) 12 19.209831953048706

mae 13 4.034059
rmse 13 8.546025
mape(>10) 13 19.141769409179688

mae 14 4.00967
rmse 14 8.471786
mape(>10) 14 19.064854085445404

mae 15 3.9889562
rmse 15 8.405006
mape(>10)

In [12]:
nb_test, pred_test = y_test_ny.shape[1], np.zeros_like(y_test_ny)
model = MC_STGCN(20, 20, 1008, ny_6community, device).to(device)
model.load_state_dict(torch.load("./saved/mc_stgcn_ny.pt"))

t0 = time.time()
with torch.no_grad():
    batched_graph1 = create_batches(X_test_ny, nb_test, edges_in_ny[:,0], edges_in_ny[:,1], 63, device)
    batched_graph2 = create_batches(X_test_ny, nb_test, ny_high_similar_poi['i'], ny_high_similar_poi['j'], 63, device)
    logits_test = model(batched_graph1, batched_graph2)
    print('Time cost: ', time.time() - t0)                        
    for j in range(len(ny_6community)):
        pred_test[ny_6community[j], :] = logits_test[j].cpu().numpy().T

rmse_test = compute_metric(y_test_ny.flatten(), pred_test.flatten(), 'TEST')

Time cost:  1.7744567394256592
mae TEST 3.658229
rmse TEST 7.704912
mape(>10) TEST 18.68145316839218
